In [ ]:
!pip3 install geojson
!pip3 install osmnx
!pip3 install shapely.constructive
import pandas as pd
import numpy as np
import csv
import plotly.express as px
import geopandas as gpd
import json
import requests
# import ipython-sql
# import regex as re
%matplotlib inline
from shapely.geometry import Point
from geopandas import datasets, GeoDataFrame, read_file
from google.colab import drive
drive.mount('/content/drive')
workdir = '/content/drive/MyDrive/Colab Notebooks/Hazus Data/' # for Tina
# workdir = '/content/drive/MyDrive/Hazus Data/'                 # for Adam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 95 kB 2.6 MB/s 
     |████████████████████████████████| 11.2 MB 23.8 MB/s 
     |████████████████████████████████| 1.0 MB 42.9 MB/s 
     |████████████████████████████████| 6.3 MB 43.7 MB/s 
     |████████████████████████████████| 1.0 MB 48.3 MB/s 
     |████████████████████████████████| 62 kB 766 kB/s 
     |████████████████████████████████| 16.7 MB 241 kB/s 
     |████████████████████████████████| 944 kB 52.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement shapely.constructive (from versions: none)
ERROR: No matching distribution found for shapely.constructive
Mounted at /content/drive


In [ ]:
def read_tables_first(state,table):
  try:
    path = f'{workdir}Raw_Data/{state}/{state}_{table}.csv'
    df = pd.read_csv(path,sep =',')
    if all(df['CensusBlock'].astype(str).str.len() == 15):
      df["County Code"] = df["CensusBlock"].astype(str).str[:5].astype(str)
      df["CensusBlock"] = df["CensusBlock"].astype(str)
    else:
      df["CensusBlock"] = df["CensusBlock"].astype(str).str.rjust(15, '0')
      df["County Code"] = df["CensusBlock"].astype(str).str[:5].astype(str)
      df["CensusBlock"] = df["CensusBlock"].astype(str)
    df.set_index('CensusBlock', inplace=True)
    df.drop('OBJECTID', axis=1, inplace=True)
    df = remove_columns_name(df, 'I')
    df = set_multiindex(df, 'BuildingCount')
  except FileNotFoundError:
    df = None
    print(f"{table}: can't find the file path")
  return df

In [ ]:
def read_tables(state,table):
  try:
    path = f'{workdir}Raw_Data/{state}/{state}_{table}.csv'
    df = pd.read_csv(path,sep =',')
    if all(df['CensusBlock'].astype(str).str.len() == 15):
      df["CensusBlock"] = df["CensusBlock"].astype(str)
    else:
      df["CensusBlock"] = df["CensusBlock"].astype(str)
      df["CensusBlock"] = df["CensusBlock"].str.rjust(15, '0')
      df['CensusBlock'] = df['CensusBlock'].astype(str)
    df.set_index('CensusBlock', inplace=True)
    df.drop('OBJECTID', axis=1, inplace=True)
  except FileNotFoundError:
    df = None
    print(f"{table}: can't find the file path")
  return df

In [ ]:
def remove_columns_name(table,word):
  cols = [col[:-1] if col.endswith(word) else col for col in table.columns]
  table.columns = cols
  return table

In [ ]:
def set_multiindex(table,group):
  MI = pd.MultiIndex.from_product([[group],table.columns], names=['group','feature'])
  table.columns = MI
  return table

In [ ]:
def Sqft_ExpC_Exp(table,word,group):
  table = remove_columns_name(table,word)
  table *=1000
  table = set_multiindex(table,group)
  return table

In [ ]:
def vehicleInv(table,group):
  cols = []
  for col in table.columns:
    if col.startswith('TVal'):
      cols.append(col)
  cols.append('TotalExp')
  table[cols] *= 1000
  table = set_multiindex(table,group)
  return table


In [ ]:
def tigerCensus(table,group):
  table.drop(['Shape','Tract', 'BlockArea','CenLat','CenLongit','IncomeRatio'],axis=1, inplace=True)
  for cols in table.iloc[:,2:]:
    table[cols] = table[cols].astype(float)*0.01
  table = set_multiindex(table,group)
  return table

In [ ]:
def treeParameter(table,group):
  for cols in table.loc[:,'TreeHeightLess40':'TreeHeightGreater60']:
    table[cols] = table[cols].astype(float)*0.01
  table = set_multiindex(table,group)
  return table

In [ ]:
def read_all_tables(state):
  hzBldgCountOccupB = read_tables_first(state,'hzBldgCountOccupB')
  # set up SqFootageOccupB
  hzSqFootageOccupB = read_tables(state,'hzSqFootageOccupB')
  if isinstance(hzSqFootageOccupB, pd.DataFrame):
    hzSqFootageOccupB = Sqft_ExpC_Exp(hzSqFootageOccupB,'F','TotalSquareFootage')

  #set up flNightVehicleInv and flDayVehicleInv
  flNightVehicleInv = read_tables(state,'flNightVehicleInv')
  if isinstance(flNightVehicleInv, pd.DataFrame):
    flNightVehicleInv = vehicleInv(flNightVehicleInv,'VehiclesNight')
  flDayVehicleInv = read_tables(state,'flDayVehicleInv')
  if isinstance(flDayVehicleInv, pd.DataFrame):
    flDayVehicleInv = vehicleInv(flDayVehicleInv,'VehiclesDay')

  # set up hzExposureOccupB and hzExposureContentOccupB
  hzExposureOccupB = read_tables(state,'hzExposureOccupB')
  if isinstance(hzExposureOccupB, pd.DataFrame):
    hzExposureOccupB = Sqft_ExpC_Exp(hzExposureOccupB,'I','ReplacementValue')
  hzExposureContentOccupB = read_tables(state,'hzExposureContentOccupB')
  if isinstance(hzExposureContentOccupB, pd.DataFrame):
    hzExposureContentOccupB = Sqft_ExpC_Exp(hzExposureContentOccupB,'I','ContentValue')

  # set up hzDemographicsB
  hzDemographicsB = read_tables(state,'hzDemographicsB')
  if isinstance(hzDemographicsB, pd.DataFrame):
    hzDemographicsB = set_multiindex(hzDemographicsB,'Demographics')

  # set up hzCensusBlock_TIGER
  hzCensusBlock_TIGER = read_tables(state,'hzCensusBlock_TIGER')
  if isinstance(hzCensusBlock_TIGER, pd.DataFrame):
    hzCensusBlock_TIGER = tigerCensus(hzCensusBlock_TIGER,'TIGERCensusBlock')

  # set up huTreeParametersB
  huTreeParametersB = read_tables(state,'huTreeParametersB')
  if isinstance(huTreeParametersB, pd.DataFrame):
    huTreeParametersB = treeParameter(huTreeParametersB,'TreeParameter')

  # set up huCensusBlock
  huCensusBlock = read_tables(state,'huCensusBlock')
  if isinstance(huCensusBlock,pd.DataFrame):
    huCensusBlock = set_multiindex(huCensusBlock, 'Hurricane')
  summary_table = pd.concat(
    [hzBldgCountOccupB,
     hzSqFootageOccupB,
     flNightVehicleInv,
     flDayVehicleInv,
     hzExposureOccupB,
     hzExposureContentOccupB,
     hzDemographicsB,
     hzCensusBlock_TIGER,
     huTreeParametersB,
     huCensusBlock]
     ,
     axis=1)
  return summary_table

In [ ]:
state = 'DC'
US_FIPS = pd.read_csv(f'{workdir}US_FIPS.csv')
US_FIPS['fips'] = US_FIPS['fips'].astype(str).str.rjust(5,'0')
FIPS = US_FIPS.loc[US_FIPS["state"] == state]
FIPS

,fips,name,state
329,11001,District of Columbia,DC


In [ ]:
# state_file = gpd.read_file(f'{workdir}tl_2010_us_county10.zip')
# state_file

In [ ]:
# PR = pd.read_csv(f'{workdir}PR.csv')
# PR

In [ ]:
# PR['FIPS'] = PR['FIPS'].astype(str)
# PR['CC'] = PR['CC'].astype(str).str.rjust(3,'0')
# PR['code'] = PR['FIPS']+PR['CC']
# PR

In [ ]:
sum = read_all_tables(state)
sum

group           BuildingCount                                                \
feature                  RES1 RES2 RES3A RES3B RES3C RES3D RES3E RES3F RES4   
CensusBlock                                                                   
110010001001000          73.0  0.0   7.0   4.0   1.0   2.0   1.0   1.0  0.0   
110010001001001           5.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
110010001001002          25.0  0.0   2.0   1.0   0.0   1.0   0.0   0.0  0.0   
110010001001003          29.0  0.0   3.0   2.0   0.0   1.0   0.0   1.0  0.0   
110010001001004          20.0  0.0   2.0   1.0   0.0   0.0   0.0   0.0  0.0   
...                       ...  ...   ...   ...   ...   ...   ...   ...  ...   
110010111003050           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
110010111003051           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
110010111003052           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
110010111003054           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
110010111003057           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   

group                 ... TreeParameter                                    \
feature         RES5  ...  StemsPerAcre TreeHeightLess40 TreeHeight40To60   
CensusBlock           ...                                                   
110010001001000  1.0  ...            82             0.23             0.31   
110010001001001  0.0  ...            51             0.23             0.31   
110010001001002  0.0  ...            31             0.23             0.31   
110010001001003  0.0  ...            30             0.23             0.31   
110010001001004  0.0  ...            50             0.23             0.31   
...              ...  ...           ...              ...              ...   
110010111003050  0.0  ...            59             0.23             0.31   
110010111003051  0.0  ...           121             0.23             0.31   
110010111003052  0.0  ...            14             0.23             0.31   
110010111003054  0.0  ...            10             0.23             0.31   
110010111003057  0.0  ...            15             0.23             0.31   

group                                                            Hurricane  \
feature         TreeHeightGreater60 TreeCollectionFactor  huBldgSchemeName   
CensusBlock                                                                  
110010001001000                0.46              0.38834  Northeast_Inland   
110010001001001                0.46              1.00000  Northeast_Inland   
110010001001002                0.46              1.00000  Northeast_Inland   
110010001001003                0.46              1.00000  Northeast_Inland   
110010001001004                0.46              1.00000  Northeast_Inland   
...                             ...                  ...               ...   
110010111003050                0.46              0.08896  Northeast_Inland   
110010111003051                0.46              1.00000  Northeast_Inland   
110010111003052                0.46              0.07741  Northeast_Inland   
110010111003054                0.46              0.20424  Northeast_Inland   
110010111003057                0.46              0.40938  Northeast_Inland   

group                                                                          
feature         huOccMapSchemeName DistToCoast SURFACEROUGHNESS WindGridIndex  
CensusBlock                                                                    
110010001001000   Northeast_Inland   30.000000            0.607             1  
110010001001001   Northeast_Inland   30.000000            0.575             1  
110010001001002   Northeast_Inland   30.000000            0.585             1  
110010001001003   Northeast_Inland   30.000000            0.582             1  
110010001001004   Northeast_Inland   30.000000            0.540             1  
...                            ...         ...              ...           ...  
1

In [ ]:
for i in FIPS["fips"]:
  row = FIPS.loc[FIPS["fips"] == i].values[0]
  code = row[0]
  name = row[1]
  table = sum.copy()
  state_sum = table.loc[table[('BuildingCount','County Code')] == code]
  state_sum = state_sum.drop(('BuildingCount','County Code'),axis = 1)
  table = sum.copy()
  name = name.replace(" ", "_")
  state_sum.to_csv(f'{workdir}Processed_Data/{state}/{state}_{code}_{name}.csv')

In [ ]:
hzBldgCountOccupB = read_tables_first(state,'hzBldgCountOccupB')
  # set up SqFootageOccupB
hzSqFootageOccupB = read_tables(state,'hzSqFootageOccupB')
hzSqFootageOccupB = Sqft_ExpC_Exp(hzSqFootageOccupB,'F','TotalSquareFootage')

  #set up flNightVehicleInv and flDayVehicleInv
flNightVehicleInv = read_tables(state,'flNightVehicleInv')
flNightVehicleInv = vehicleInv(flNightVehicleInv,'VehiclesNight')
flDayVehicleInv = read_tables(state,'flDayVehicleInv')
flDayVehicleInv = vehicleInv(flDayVehicleInv,'VehiclesDay')

  # set up hzExposureOccupB and hzExposureContentOccupB
hzExposureOccupB = read_tables(state,'hzExposureOccupB')
hzExposureOccupB = Sqft_ExpC_Exp(hzExposureOccupB,'I','ReplacementValue')
hzExposureContentOccupB = read_tables(state,'hzExposureContentOccupB')
hzExposureContentOccupB = Sqft_ExpC_Exp(hzExposureContentOccupB,'I','ContentValue')

  # set up hzDemographicsB
hzDemographicsB = read_tables(state,'hzDemographicsB')
hzDemographicsB = set_multiindex(hzDemographicsB,'Demographics')

  # set up hzCensusBlock_TIGER
hzCensusBlock_TIGER = read_tables(state,'hzCensusBlock_TIGER')
hzCensusBlock_TIGER = tigerCensus(hzCensusBlock_TIGER,'TIGERCensusBlock')

  # set up huTreeParametersB
huTreeParametersB = read_tables(state,'huTreeParametersB')
huTreeParametersB = treeParameter(huTreeParametersB,'TreeParameter')

  # set up huCensusBlock
huCensusBlock = read_tables(state,'huCensusBlock')
huCensusBlock = set_multiindex(huCensusBlock, 'Hurricane')